In [105]:
!python -V

Python 3.9.12


In [106]:
import pandas as pd

In [107]:
import pickle

In [108]:
import seaborn as sns
import matplotlib.pyplot as plt

In [109]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [110]:
!pip install pyarrow

In [111]:
!ls ../../notebooks/data/

fhv_tripdata_2021-01.parquet  fhv_tripdata_2021-02.parquet


In [112]:
df = pd.read_parquet('../../notebooks/data/fhv_tripdata_2021-01.parquet')


In [132]:
df.columns

Index(['dispatching_base_num', 'pickup_datetime', 'dropOff_datetime',
       'PUlocationID', 'DOlocationID', 'SR_Flag', 'Affiliated_base_number',
       'duration'],
      dtype='object')

In [114]:

df['duration'] = df.dropOff_datetime - df.pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)


In [115]:
df.duration.mean()

19.1672240937939

In [116]:
df[~((df.duration >= 1) & (df.duration <= 60))].shape[0] 

44286

In [117]:

df = df[(df.duration >= 1) & (df.duration <= 60)]


In [118]:
df['PUlocationID'] = df['PUlocationID'].fillna('-1')
df['DOlocationID'] = df['DOlocationID'].fillna('-1')

/tmp/ipykernel_9008/2743488764.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PUlocationID'] = df['PUlocationID'].fillna('-1')
/tmp/ipykernel_9008/2743488764.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DOlocationID'] = df['DOlocationID'].fillna('-1')


In [119]:
df[df['PUlocationID']=='-1'].shape[0] / df.shape[0]

0.8352732770722617

In [120]:

categorical = ['PUlocationID', 'DOlocationID']


In [121]:
X_train = pd.get_dummies(df[categorical], drop_first=True)

In [122]:
X_train

,PUlocationID_2.0,PUlocationID_3.0,PUlocationID_4.0,PUlocationID_5.0,PUlocationID_6.0,PUlocationID_7.0,PUlocationID_8.0,PUlocationID_9.0,PUlocationID_10.0,PUlocationID_11.0,...,DOlocationID_257.0,DOlocationID_258.0,DOlocationID_259.0,DOlocationID_260.0,DOlocationID_261.0,DOlocationID_262.0,DOlocationID_263.0,DOlocationID_264.0,DOlocationID_265.0,DOlocationID_-1
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154107,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1154108,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1154109,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1154110,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [123]:

y_train = df['duration']
#y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

10.528519107168242

In [124]:
df_val = pd.read_parquet('../../notebooks/data/fhv_tripdata_2021-02.parquet')

In [125]:

df_val['duration'] = df_val.dropOff_datetime - df_val.pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)


In [126]:
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]


In [127]:
df_val['PUlocationID'] = df_val['PUlocationID'].fillna('-1')
df_val['DOlocationID'] = df_val['DOlocationID'].fillna('-1')

In [138]:
X_val = pd.get_dummies(df_val[categorical], drop_first=True)

In [139]:
X_val

,PUlocationID_2.0,PUlocationID_3.0,PUlocationID_4.0,PUlocationID_5.0,PUlocationID_6.0,PUlocationID_7.0,PUlocationID_8.0,PUlocationID_9.0,PUlocationID_10.0,PUlocationID_11.0,...,DOlocationID_257.0,DOlocationID_258.0,DOlocationID_259.0,DOlocationID_260.0,DOlocationID_261.0,DOlocationID_262.0,DOlocationID_263.0,DOlocationID_264.0,DOlocationID_265.0,DOlocationID_-1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1037687,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1037688,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1037689,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1037690,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [129]:
y_val = df_val.duration

In [140]:
X_val=X_val.drop('PUlocationID_2.0', axis=1)

In [141]:
#lr = LinearRegression()
#lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

/home/ubuntu/anaconda3/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- DOlocationID_110.0
Feature names seen at fit time, yet now missing:
- PUlocationID_2.0

  warnings.warn(message, FutureWarning)


16.095593753895248